In [73]:
#imports and call stored df
import pandas as pd


%store -r cb_df_final
%store -r foursquare_df_final
%store -r yelp_df_final

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [74]:
#combine individually with cb then together
cb_foursquare = cb_df_final.merge(foursquare_df_final, how='outer', left_on=['latitude', 'longitude'], right_on=['latitude', 'longitude'])
cb_yelp = cb_df_final.merge(yelp_df_final, how='outer', left_on=['latitude', 'longitude'], right_on=['latitude', 'longitude'])
#combined_df_final = cb_foursquare.merge(yelp_df_final, how='outer', left_on=['latitude', 'longitude', 'name', 'distance'], right_on=['latitude', 'longitude', 'name', 'distance'])
combined_df_final = pd.concat([cb_foursquare, cb_yelp], ignore_index=True)
combined_df_final


,latitude,longitude,free_bikes,distance,name,popularity,price,rating,address,total_photos,total_ratings,total_tips
0,49.262487,-123.114397,8.0,779.000000,The Juice Truck,0.912545,1.0,9.1,"28 5th Ave W (btwn Manitoba & Ontario), Vancou...",26.0,38.0,3.0
1,49.274566,-123.121817,6.0,286.000000,The Keg Steakhouse + Bar - Yaletown,0.973651,3.0,8.2,"1011 Mainland St (at Nelson St), Vancouver BC ...",249.0,210.0,59.0
2,49.279764,-123.110154,16.0,512.000000,Pourhouse Restaurant,0.970276,2.0,9.0,"162 Water St (btwn Cambie & Abbott), Vancouver...",343.0,380.0,118.0
3,49.260599,-123.113504,6.0,618.000000,Rain or Shine,0.977197,1.0,8.6,"3382 Cambie St (18th Ave), Vancouver BC V5Z 2W5",95.0,78.0,13.0
4,49.264215,-123.117772,7.0,921.000000,The Juice Truck,0.912545,1.0,9.1,"28 5th Ave W (btwn Manitoba & Ontario), Vancou...",26.0,38.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
485,49.250940,-123.101306,3.0,1281.920753,V Yaletown,NaN,$$,3.5,"[1095 Mainland Street, Upper Level, Vancouver,...",NaN,6.0,NaN
486,49.280977,-123.035969,15.0,392.783241,McDonald's,NaN,$,2.0,"[3444 East Hastings Street, Vancouver, BC V5K ...",NaN,33.0,NaN
487,49.270783,-123.141564,18.0,758.360382,Earls Kitchen + Bar,NaN,$$$,3.0,"[1601 West Broadway, Vancouver, BC V6J 1W9, Ca...",NaN,190.0,NaN
488,49.264019,-123.209176,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.